In [3]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

from ailab_tools.utils import split_and_stack

zip解凍

In [ ]:
#zipしたファイル中身(数)確認
for i in range(1,5):
    zf = zipfile.ZipFile(f"/home/yusuke_akada/notebook/Drawbridge/xd{i}.zip", 'r')
    print(zf.namelist())

In [ ]:
#zipしたファイルの解凍
for i in range(1,5):
    with zipfile.ZipFile(f"/home/yusuke_akada/notebook/Drawbridge/xd{i}.zip", 'r') as existing_zip:
        existing_zip.extractall('/home/yusuke_akada/notebook/Drawbridge')

味見　中身の確認

In [ ]:
df = pd.read_csv("/home/yusuke_akada/notebook/Drawbridge/part-00001_after.txt", header=None, names=['column'])
df.head()

In [ ]:
splitted=df['column'].str.split(":",expand=True)
splitted.columns = ['A', 'B']
del splitted['A']
splitted=splitted['B'].str.split("|",1,expand=True)
splitted.columns = ['drawbridge_consumer_id', 'smn_uid']
stacked = split_and_stack(splitted, "smn_uid", sep="|")
stacked.head()

分割されたテキストファイルを縦持ち＆統合しImpalaにUp

In [ ]:
%%slack_notify @yusuke_akada
uid_table = "segment.drawbridge_1901"   # tapad_idとsmn_uidの最終的な対応table
temp_table = "dm_tmp.ya_temp_drawbridge_190131"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            drawbridge_consumer_id string,
            smn_uid bigint
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query)    

    # テキストファイル単体ごとに読込、DataFrame化
    for i in range(0,200):
            if i < 10:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/Drawbridge/part-0000{i}_after.txt", header=None, names=['column'])
            elif 10 <= i < 100:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/Drawbridge/part-000{i}_after.txt", header=None, names=['column'])
            else:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/Drawbridge/part-00{i}_after.txt", header=None, names=['column'])
                
            splitted=df['column'].str.split(":",expand=True)
            splitted.columns = ['A', 'B']
            del splitted['A']
            splitted=splitted['B'].str.split("|",1,expand=True)
            splitted.columns = ['drawbridge_consumer_id', 'smn_uid']
            stacked = split_and_stack(splitted, "smn_uid", sep="|")
        
        # ImpalaにUP
            ir.upload_dataframe(temp_table, stacked, ['string','bigint'])
        
        # 親テーブルにInsert
            query = (
                f"""
                insert overwrite table {uid_table}
                    (
                    drawbridge_consumer_id,
                    smn_uid 
                    )

                partition
                    (
                    block = {i}
                    )

                select
                 drawbridge_consumer_id,
                 smn_uid
                from {temp_table}
                """
            )
            ir.execute_query_no_ret(query) 

            # tempテーブルdrop
            ir.drop_tables([temp_table])